In [ ]:
from beamtest_analysis_helper import DecodeBinary
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path
from natsort import natsorted
import pandas as pd

In [ ]:
def decode_directory(path_to_dir):
    # Assuming each directory has a list of files to decode
    files = sorted(list(path_to_dir.glob('TDC*bin')))
    decoder = DecodeBinary(
        firmware_key=0b0001,
        board_id=[0x17f0f, 0x17f0f, 0x17f0f, 0x17f0f],
        file_list=files,
        save_nem=None,
        skip_fw_filler=True,
        skip_event_df=True,
        skip_crc_df=True,
    )
    df, _, _, filler_df = decoder.decode_files()

    return df, filler_df

In [ ]:
dirs = natsorted(list(Path('/home/jongho/Physics/ETROC/ETROC-Analysis/binary_files').glob('ET2p03_BARE9*Threshold*')))
dirs

In [ ]:
with ProcessPoolExecutor() as executor:
    results = list(executor.map(decode_directory, dirs))

In [ ]:
dfs = []

for idx, idir in enumerate(dirs):
    info = idir.name.split('_')
    thres = int(info[-1])
    charge = int(info[-3])

    tmp = results[idx][0][1000:].copy()

    if tmp.shape[0] < 100:
        continue

    tmp.loc[:, 'charge'] = charge
    tmp.loc[:, 'threshold'] = thres

    tmp['charge'] = tmp['charge'].astype('uint8')
    tmp['threshold'] = tmp['threshold'].astype('uint16')

    dfs.append(tmp)

In [ ]:
df = pd.concat(dfs)
del dfs

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df

In [ ]:
df.to_feather('ET2p03_BARE9_Run1_externalVRef1V_qinj_moneyplot.feather')

In [ ]:
### filler
# results[0][1]

In [ ]:
### Drop unnecessary columns
df.drop(columns=['ea', 'bcid', 'l1a_counter'], inplace=True)

In [ ]:
cal_table = df.pivot_table(index=["row", "col", "charge", "threshold"], columns=["board"], values=["cal"], aggfunc=lambda x: x.mode().iat[0])

cal_table = cal_table.reset_index().set_index([('row', ''), ('col', ''), ('charge', ''), ('threshold', '')]).stack().reset_index()
cal_table.columns = ['row', 'col', 'charge', 'threshold', 'board', 'cal_mode']

merged_df = pd.merge(df, cal_table, on=['board', 'row', 'col', 'charge','threshold'])
del df, cal_table

cal_condition = abs(merged_df['cal'] - merged_df['cal_mode']) <= 3
merged_df = merged_df[cal_condition].drop(columns=['cal_mode'])

cal_filtered_df = merged_df.reset_index(drop=True)
cal_filtered_df['board'] = cal_filtered_df['board'].astype('uint8')
del merged_df, cal_condition

In [ ]:
tot_table = cal_filtered_df.pivot_table(index=["row", "col", "charge", "threshold"], columns=["board"], values=["tot"], aggfunc=lambda x: x.mode().iat[0])

tot_table = tot_table.reset_index().set_index([('row', ''), ('col', ''), ('charge', ''), ('threshold', '')]).stack().reset_index()
tot_table.columns = ['row', 'col', 'charge', 'threshold', 'board', 'tot_mode']

merged_df = pd.merge(cal_filtered_df, tot_table, on=['board', 'row', 'col', 'charge','threshold'])
del cal_filtered_df, tot_table

tot_condition = abs(merged_df['tot'] - merged_df['tot_mode']) <= 100
merged_df = merged_df[tot_condition].drop(columns=['tot_mode'])

tot_filtered_df = merged_df.reset_index(drop=True)
tot_filtered_df['board'] = tot_filtered_df['board'].astype('uint8')
del merged_df, tot_condition

In [ ]:
tot_filtered_df

In [ ]:
grouped = tot_filtered_df.groupby(['row', 'col', 'charge', 'threshold'])
agg_df = grouped.agg(
    cal_mean=('cal', 'mean'),
    cal_std=('cal', 'std'),
    toa_mean = ('toa','mean'),
    toa_std = ('toa', 'std'),
    tot_mean=('tot', 'mean'),
    tot_std=('tot', 'std'),
)
# agg_df.to_pickle(f'{board_name}_{run_name}_TDC_Summary.pickle')

In [ ]:
agg_df

In [ ]:
import matplotlib.pyplot as plt
import mplhep as hep
hep.style.use('CMS')

In [ ]:
def draw_cal(pixels, input_df, board_name, run_name):
    for (row, col) in pixels:
        subset = input_df.xs((row, col), level=('row', 'col'))

        if subset.empty:
            continue

        charges = subset.index.get_level_values('charge').unique()

        fig, axes = plt.subplots(1, 2, figsize=(22, 10))

        for charge in charges:
            charge_data = subset[['cal_mean', 'cal_std']].xs(charge, level='charge')
            charge_data = charge_data[(charge_data['cal_mean'] < 300.)]
            axes[0].plot(charge_data.index.get_level_values('threshold'), charge_data['cal_mean'], '.-', label=f'{charge} fC')
            axes[1].plot(charge_data.index.get_level_values('threshold'), charge_data['cal_std'], '.-', label=f'{charge} fC')

        hep.cms.text(loc=0, ax=axes[0], text="ETL ETROC", fontsize=18)
        hep.cms.text(loc=0, ax=axes[1], text="ETL ETROC", fontsize=18)
        axes[0].set_title(f'Row {row}, Col {col}', loc='right', fontsize=16)
        axes[1].set_title(f'Row {row}, Col {col}', loc='right', fontsize=16)

        axes[0].set_ylim(charge_data['cal_mean'].mean()-5, charge_data['cal_mean'].mean()+5)
        axes[1].set_ylim(-0.03, 1)

        axes[0].set_xlabel('Threshold')
        axes[1].set_xlabel('Threshold')
        axes[0].set_ylabel('CAL Mean')
        axes[1].set_ylabel('CAL Std')
        axes[0].legend()
        axes[1].legend()

        plt.tight_layout()
        # fig.savefig(f'{board_name}/{board_name}_{run_name}_Row_{row}_Col_{col}_CAL.png')
        # fig.savefig(f'{board_name}/{board_name}_{run_name}_Row_{row}_Col_{col}_CAL.pdf')
        # plt.close(fig)

In [ ]:
def draw_toa(pixels, input_df, board_name, run_name):
    for (row, col) in pixels:
        subset = input_df.xs((row, col), level=('row', 'col'))

        if subset.empty:
            continue

        charges = subset.index.get_level_values('charge').unique()

        fig, axes = plt.subplots(1, 2, figsize=(22, 10))

        for charge in charges:
            charge_data = subset[['toa_mean', 'toa_std']].xs(charge, level='charge')
            charge_data = charge_data[(charge_data['toa_mean'] < 300.) & (charge_data['toa_std'] < 3.)]
            axes[0].plot(charge_data.index.get_level_values('threshold'), charge_data['toa_mean'], '.-', label=f'{charge} fC')
            axes[1].plot(charge_data.index.get_level_values('threshold'), charge_data['toa_std'], '.-', label=f'{charge} fC')

        hep.cms.text(loc=0, ax=axes[0], text="ETL ETROC", fontsize=18)
        hep.cms.text(loc=0, ax=axes[1], text="ETL ETROC", fontsize=18)
        axes[0].set_title(f'Row {row}, Col {col}', loc='right', fontsize=16)
        axes[1].set_title(f'Row {row}, Col {col}', loc='right', fontsize=16)

        axes[1].set_ylim(-0.03, 4)
        # axes[0].set_ylim(200, 290)

        axes[0].set_xlabel('Threshold')
        axes[1].set_xlabel('Threshold')
        axes[0].set_ylabel('TOA Mean')
        axes[1].set_ylabel('TOA Std')
        axes[0].legend()
        axes[1].legend()

        plt.tight_layout()
        # fig.savefig(f'{board_name}/{board_name}_{run_name}_Row_{row}_Col_{col}_TOA.png')
        # fig.savefig(f'{board_name}/{board_name}_{run_name}_Row_{row}_Col_{col}_TOA.pdf')
        # plt.close(fig)

In [ ]:
def draw_tot(pixels, input_df, board_name, run_name):
    for (row, col) in pixels:
        subset = input_df.xs((row, col), level=('row', 'col'))

        if subset.empty:
            continue

        charges = subset.index.get_level_values('charge').unique()

        fig, axes = plt.subplots(1, 2, figsize=(22, 10))

        for charge in charges:
            charge_data = subset[['tot_mean', 'tot_std']].xs(charge, level='charge')
            charge_data = charge_data[(charge_data['tot_mean'] < 300.) & (charge_data['tot_std'] < 3.) & (charge_data['tot_std'] != 0)]
            axes[0].plot(charge_data.index.get_level_values('threshold'), charge_data['tot_mean'], '.-', label=f'{charge} fC')
            axes[1].plot(charge_data.index.get_level_values('threshold'), charge_data['tot_std'], '.-', label=f'{charge} fC')

        hep.cms.text(loc=0, ax=axes[0], text="ETL ETROC", fontsize=18)
        hep.cms.text(loc=0, ax=axes[1], text="ETL ETROC", fontsize=18)
        axes[0].set_title(f'Row {row}, Col {col}', loc='right', fontsize=16)
        axes[1].set_title(f'Row {row}, Col {col}', loc='right', fontsize=16)

        axes[1].set_ylim(-0.03, 4)

        axes[0].set_xlabel('Threshold')
        axes[1].set_xlabel('Threshold')
        axes[0].set_ylabel('TOT Mean')
        axes[1].set_ylabel('TOT Std')
        axes[0].legend()
        axes[1].legend()

        plt.tight_layout()
        # fig.savefig(f'{board_name}/{board_name}_{run_name}_Row_{row}_Col_{col}_TOT.png')
        # fig.savefig(f'{board_name}/{board_name}_{run_name}_Row_{row}_Col_{col}_TOT.pdf')
        # plt.close(fig)

In [ ]:
row_col_combinations = [(0, 0), (0, 14), (3, 0), (3, 14), (12, 0), (12, 14)]

In [ ]:
b_name = ''
r_name = ''

draw_cal(row_col_combinations, agg_df, b_name, r_name)
draw_toa(row_col_combinations, agg_df, b_name, r_name)
draw_tot(row_col_combinations, agg_df, b_name, r_name)